In [ ]:
# useful article about OHLC data aggregation:
# applicable to websocket streams of tick by tick data
# https://blog.quantinsti.com/tick-tick-ohlc-data-pandas-tutorial/


In [ ]:
# jupyter did not want to load already installed talib library 
# so had to reinstall it via anaconda as well 
#!conda install -c conda-forge ta-lib
import talib as ta
import yfinance as yf
import pandas as pd
import sqlite3

In [ ]:
# custom function imports
from functions_gen import *        # general functions
from functions_ml import *         # machine learning
from functions_viz import *        # visualization
from functions_db import *         # database

# custom indicators moved to modules
from functions_superjump import *
from functions_HHLL import *
from functions_HHLL_conf import *
from functions_HHLL_channel import *
from functions_gator import *

# Random Forest specific functions
from functions_forest import *

In [ ]:
from configparser import ConfigParser

In [ ]:
config = ConfigParser()
config.read("config.ini")
API_KEY = config.get("alpaca", "API_KEY")
SECRET_KEY = config.get("alpaca", "SECRET_KEY")

print(f"API_KEY:    ", API_KEY)
print(f"SECRET_KEY: ", SECRET_KEY)

In [ ]:
# sqlite database structure is following:
#
#sqlite> .header on
#sqlite> .mode column
#sqlite> select * from alpaca_websocket_stream_data LIMIT 10;
#timestamp                            symbol  price   size  exchange  conditions  tape  id   
#-----------------------------------  ------  ------  ----  --------  ----------  ----  -----
#2022-07-19 15:49:25.477387108-04:00  AAPL    150.8   100   V         ['@']       C     10807
#32022-07-19 15:49:27.252579851-04:00  AAPL    150.81  3     V         ['@', 'I']  C     10808
#2022-07-19 15:49:27.252579851-04:00  AAPL    150.81  100   V         ['@']       C     10809
#2022-07-19 15:49:27.666163652-04:00  AAPL    150.81  100   V         ['@']       C     10810
#2022-07-19 15:49:27.666164795-04:00  AAPL    150.81  200   V         ['@']       C     10811
#2022-07-19 15:49:29.248316808-04:00  AAPL    150.79  100   V         ['@']       C     10812
#2022-07-19 15:49:32.963910211-04:00  AAPL    150.78  35    V         ['@', 'I']  C     10813
#2022-07-19 15:49:36.611092454-04:00  AAPL    150.77  2     V         ['@', 'I']  C     10814
#2022-07-19 15:49:36.612940345-04:00  AAPL    150.77  100   V         ['@']       C     10815
#2022-07-19 15:49:37.083678369-04:00  AAPL    150.76  100   V         ['@']       C     10816
#sqlite> 





# Exploratory data wrangling
Optionally uncomment the code to get insights to individual steps.
Can be useful for db troubleshooting later

In [ ]:
# connect to sqlite database and get all data where symbol is AAPL
# symbol is external variable
# pandas to onnect to database and aggregate price data to 1 minute granularity in pandas

#symbol='AAPL'
#
#conn = sqlite3.connect("alpaca_websocket_stream_data.db")
#c = conn.cursor()
#c.execute("SELECT * FROM alpaca_websocket_stream_data WHERE symbol = ?", (symbol,))
#data = c.fetchall()
#conn.close()

In [ ]:
#data

In [ ]:
# LOAD ONE DAY BACK

# connect to sqlite database and get all data where symbol is AAPL
# timestamp is from 24 hours ago to now
#conn = sqlite3.connect("alpaca_websocket_stream_data.db")
#c = conn.cursor()
#c.execute("SELECT * FROM alpaca_websocket_stream_data WHERE symbol = 'AAPL' AND timestamp BETWEEN datetime('now', '-1 month') AND datetime('now')")
#data = c.fetchall()
#conn.close()

In [ ]:
#data

In [ ]:
# load data to dataframe
#df = pd.DataFrame(data, columns=["timestamp", "symbol", "price", "size", "exchange", "conditions", "tape", "id"])

In [ ]:
#df

In [ ]:
#df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ns")

In [ ]:
#df["date"] = df["timestamp"].dt.date

In [ ]:
#df

In [ ]:
#df = df.set_index("timestamp")


In [ ]:
#df_resampled = df['price'].resample("1Min").ohlc(_method='ohlc')

In [ ]:
#df_resampled

In [ ]:
#df = df.reset_index()
#df = df.sort_values(by=["symbol"])

## Get data from Alpaca (historical, not socket stream)

In [ ]:
get_alpaca_hist_data("BTCUSD", "crypto", API_KEY, SECRET_KEY)

In [ ]:
get_alpaca_hist_data("AAPL", "stocks", API_KEY, SECRET_KEY)

# Making function flow
one function for getting the data for specific ticker from the database
another function to process the df into resampled df with 1 and 5 min granularity

In [ ]:
symbol = 'AAPL'
db_name = 'alpaca_websocket_stream_data.db'
table_name= 'alpaca_websocket_stream_data'
granularity = '1Min'
interval = "1m"  # for yahoo finance model training if needed 

In [ ]:
# gets data only for last n days for given ticker
#data =  get_ticker_data_from_db_days_back(symbol, db_name, table_name)

In [ ]:
#data

In [ ]:
# gets all data from db for given ticker
data =  get_ticker_data_from_db(symbol, db_name, table_name)

In [ ]:
data

In [ ]:
# resample tick by tick data from db to minute timeframe and save to df
df_db =  resample_data(data, granularity=granularity)

In [ ]:
df_db

## Model training (Random Forest)

In [ ]:
# not needed so far, we can import pretrained model

In [ ]:
# but best to allow for training here as well

In [ ]:
def training_sequence(tickers, interval="1m", model_name="./random_forest.joblib"):
    # initiates training sequence for random forest classifier

    for ticker in tickers:
        print('ticker: ', ticker)
        df = get_data(ticker, interval)
        plot_train_data(df, ticker)


        #print(df)

        # custom indicator extension:
        # create extra features from new indicators into new dfs
        # and then join the dfs based on minute datetime with original df
        # our model also needs 1/0 instead of True/False
        # thrend_conf col needs conversion from 'u','d' to 1,0
        out_df1 = superjumpTBB(df)      # superjumpTBB
        out_df1.replace({False: 0, True: 1}, inplace=True)

        out_df2 = HHLL_Strategy(df)  # HHHL indicator
        out_df2.replace({False: 0, True: 1}, inplace=True)

        out_df3 = HHLL_confirmation(df)  # HHHL indicator
        out_df3.replace({'d': 0, 'u': 1, 'none': -1}, inplace=True)

        out_df4 = HHLL_Channel(df)
        
        out_df5 = rsi_strategy(df) # RSI gator indicator


        df = compute_technical_indicators(df)
        df = compute_features(df)
        df = define_target_condition(df)

        # TODO, verify that inner join is what we really need
        # merging with new dataframes with custom indicators
        df = pd.merge(df, out_df1, how='inner', on='Date')
        df = pd.merge(df, out_df2, how='inner', on='Date')
        df = pd.merge(df, out_df3, how='inner', on='Date')
        df = pd.merge(df, out_df4, how='inner', on='Date')
        df = pd.merge(df, out_df5, how='inner', on='Date')

        #print('regular df')
        #print(df)

        clf = splitting_and_training(df)

        save_model(clf, model_name)
        
        # commenting out saves time during training
        #df = predict_timeseries(df, clf)
        #plot_stock_prediction(df, ticker)

    return None

In [ ]:
# training stock data
tickers = ['SPY', 'F', 'IBM', 'GE', 'AAPL', 'ADM'] 

In [ ]:
training_sequence(tickers, interval="1m", model_name="./random_forest.joblib")

## Feature importance visualization 
(Random Forest only)

In [ ]:
# load classifier, no need to initialize the loaded_rf
clf = joblib.load("./random_forest.joblib")

In [ ]:
predictors_list = ['aboveSAR','aboveUpperBB','belowLowerBB','RSI','oversoldRSI','overboughtRSI',
                   'aboveEMA5','aboveEMA10','aboveEMA15','aboveEMA20','aboveEMA30','aboveEMA40','aboveEMA50',
                   'aboveEMA60','aboveEMA70','aboveEMA80','aboveEMA90',
                   'aboveEMA100','aboveEMA200',
                   'LongSig','ShortSig','WLongSig','WShortSig',
                   'HH','LL','HL','LH',
                   'trend_conf','HHLL_channel_sig','rsi_gator_sig'
                  ]

In [ ]:
plot_forest_feature_importances(clf, predictors_list)

# Predictions
- now we have dataframe and can compute whatever indicators we want
- and then connect it to our predictive model and even visualize in streamlit

In [ ]:
# load classifier, no need to initialize the loaded_rf
clf = joblib.load("./random_forest.joblib")

In [ ]:
# this is dataframe from db
df_db.tail(2)

#### Temporary workaround - Yahoo
Due to issue with custom indicators interacting with db data, we will take 1 minute data from yahoo for now.

In [ ]:
# temp workaround since some indicator failing
# they might be failing maybe because some NaN values in our df from database
# need to investigate in detail
df = get_data(symbol, interval)

#### Temporary workaround - Alpaca
Alternatively we can get historical data from Alpaca.

In [ ]:
df = get_alpaca_hist_data("BTCUSD", "crypto", API_KEY, SECRET_KEY)    # crypto
#df = get_alpaca_hist_data("AAPL", "stocks", API_KEY, SECRET_KEY)    # stocks 15 mins delayed with free API

In [ ]:
#pd.to_datetime(df['Date'])

In [ ]:
df

In [ ]:
df.tail(2)

In [ ]:
# --- plot whole dataframe ---
#df[['Open','High','Low','Close', 'EMA20']].plot()
df[['Open','High','Low','Close']].plot()

In [ ]:
# --- plot tail of the dataframe ---
#df[['Open','High','Low','Close']].iloc[-20:].plot()

In [ ]:
# we take df that was taken from db and resampled to minute granularity (1min, 5min)
# and we will compute various indicators for it
# including custom indicators

In [ ]:
# prepare dfs with extra indicators
out_df1 = superjumpTBB(df)      # superjumpTBB
out_df1.replace({False: 0, True: 1}, inplace=True)

out_df2 = HHLL_Strategy(df)  # HHHL indicator
out_df2.replace({False: 0, True: 1}, inplace=True)

out_df3 = HHLL_confirmation(df)  # HHLL confirmation
# converting 'u','d', 'none' to integers for 'trend_conf' col
out_df3.replace({'d': 0, 'u': 1, 'none': -1}, inplace=True)

out_df4 = HHLL_Channel(df)

out_df5 = rsi_strategy(df)

In [ ]:
# compute general indicators, features and target
df = compute_technical_indicators(df)
df = compute_features(df)
df = define_target_condition(df)

In [ ]:
# merge with custom indicators
df = pd.merge(df, out_df1, how='inner', on='Date')
df = pd.merge(df, out_df2, how='inner', on='Date')
df = pd.merge(df, out_df3, how='inner', on='Date')
df = pd.merge(df, out_df4, how='inner', on='Date')
df = pd.merge(df, out_df5, how='inner', on='Date')

In [ ]:
df.tail(2)

In [ ]:
# actual prediction
# can take longer if the dataframe is big
predict_timeseries(df, clf)

### General prediction plots
does not work well for stocks with intra-day data (for example 1 minute granularity)

In [ ]:
# high level view
ticks_back = 1200 # lookback n candles from now
plot_stock_prediction_tb(df, symbol, ticks_back)

In [ ]:
# zoomed in view
ticks_back = 500 # lookback n candles from now
plot_stock_prediction_tb(df, symbol, ticks_back)

### Intra day plots
good for stocks on 1 minute time granularity, manages intra day gaps in plots

In [ ]:
# high level view
ticks_back = 1200 # lookback n candles from now
plot_stock_prediction_zoom(df, symbol, ticks_back)

In [ ]:
# zoomed in view
ticks_back = 500 # lookback n candles from now
plot_stock_prediction_zoom(df, symbol, ticks_back)

# Daemonize the script to run every minute
but running script every minute by cron should be more reliable, also prevents time drift

In [ ]:
# --- UNCOMMENT TO RUN CONTINUOUSLY ---

#while True:
#    
#    ## #data = get_ticker_data_from_db(symbol, db_name, table_name)
#    ##data = get_ticker_data_from_db_days_back(symbol, db_name, table_name)
#    ##df = resample_data(data, granularity=granularity)
#   
#
#    # yfinance temp workaround
#    df = get_data(symbol, interval)
#
#    # prepare dfs with extra indicators
#    out_df1 = superjumpTBB(df)      # superjumpTBB
#    out_df1.replace({False: 0, True: 1}, inplace=True)
#    out_df2 = HHLL_Strategy(df)  # HHHL indicator
#    out_df2.replace({False: 0, True: 1}, inplace=True)
#    out_df3 = HHLL_confirmation(df)  # HHLL confirmation
#    # converting 'u','d', 'none' to integers for 'trend_conf' col
#    out_df3.replace({'d': 0, 'u': 1, 'none': -1}, inplace=True)
#    out_df4 = HHLL_Channel(df)
#    out_df5 = rsi_strategy(df)
#
#    # compute general indicators, features and target
#    df = compute_technical_indicators(df)
#    df = compute_features(df)
#    df = define_target_condition(df)
#
#    # merge with custom indicators
#    df = pd.merge(df, out_df1, how='inner', on='Date')
#    df = pd.merge(df, out_df2, how='inner', on='Date')
#    df = pd.merge(df, out_df3, how='inner', on='Date')
#    df = pd.merge(df, out_df4, how='inner', on='Date')
#    df = pd.merge(df, out_df5, how='inner', on='Date')
#
#    # actual prediction
#    # can take longer if the dataframe is big
#    # so we are making shorter dataframe for this
#    df_short = df.iloc[-100:]
#    
#    predict_timeseries(df_short, clf)
#
#    # zoomed in plot view
#    ticks_back = 500 # lookback n candles from now
#    plot_stock_prediction_zoom(df_short, symbol, ticks_back)
#
#
#
#    time.sleep(60)
